In [1]:
! pip install --upgrade langchain chromadb langchain-experimental # (newest versions required for multi-modal)
! pip install pdf2image
! pip install pytesseract
! yes | sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr
!pip install -U langchain transformers bitsandbytes accelerate
!pip install -q sentence-transformers
!pip install -U langchain-community
!pip install rapidocr-onnxruntime
!conda install -c conda-forge poppler -y
# lock to 0.10.19 due to a persistent bug in more recent versions
! pip install "unstructured[all-docs]==0.10.19" pillow pydantic lxml open_clip_torch torch
!pip install -q PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from PIL import Image
import shutil
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
os.mkdir("/kaggle/working/LLM_RAG_Data")
os.mkdir("/kaggle/working/LLM_RAG_Data/Data")
path='/kaggle/working/LLM_RAG_Data/Data'
! wget -p -k "https://assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf"
shutil.move("assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf",os.path.join("/kaggle/working/LLM_RAG_Data/Data","ConceptsofBiology.pdf"))

--2024-06-03 08:03:30--  https://assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf
Resolving assets.openstax.org (assets.openstax.org)... 143.204.29.105, 143.204.29.40, 143.204.29.102, ...
Connecting to assets.openstax.org (assets.openstax.org)|143.204.29.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153179709 (146M) [application/pdf]
Saving to: 'assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf'

assets.openstax.org 100%[===================>] 146.08M  89.5MB/s    in 1.6s    

2024-06-03 08:03:32 (89.5 MB/s) - 'assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf' saved [153179709/153179709]

FINISHED --2024-06-03 08:03:32--
Total wall clock time: 1.8s
Downloaded: 1 files, 146M in 1.6s (89.5 MB/s)
Converted links in 0 files in 0 seconds.


'/kaggle/working/LLM_RAG_Data/Data/ConceptsofBiology.pdf'

In [3]:
!pip install PyPDF2
from PyPDF2 import PdfReader, PdfWriter
path='/kaggle/working/LLM_RAG_Data/Data/'
file_name = os.listdir(path)
writer = PdfWriter()
# def reset_eof_of_pdf_return_stream(pdf_stream_in:list):
#     # find the line position of the EOF
#     for i, x in enumerate(pdf_stream_in[::-1]):
#         if b'%%EOF' in x:
#             actual_line = len(pdf_stream_in)-i
#             print(f'EOF found at line position {-i} = actual {actual_line}, with value {x}')
#             break

#     # return the list up to that point
#     return pdf_stream_in[:actual_line]

# # opens the file for reading
# with open(path+file_name[0], 'rb') as p:
#     txt = (p.readlines())

# # get the new list terminating correctly
# txtx = reset_eof_of_pdf_return_stream(txt)

# # write to new pdf
# with open(path+file_name[0], 'wb') as f:
#     f.writelines(txtx)
    
with open(path+file_name[0], 'rb') as infile:
    for page in range(210,258):
        reader = PdfReader(infile)
        writer.add_page(reader.pages[page])
with open(path+file_name[0], 'wb') as outfile:
    writer.write(outfile)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00a 0:00:01m


In [4]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import tempfile
os.mkdir("/kaggle/working/LLM_RAG_Data/Raw_Data/")
path='/kaggle/working/LLM_RAG_Data/Raw_Data/'
path1='/kaggle/working/LLM_RAG_Data/Data/'
with tempfile.TemporaryDirectory() as path2:
    images_from_path = convert_from_path(path1+file_name[0], output_folder=path,fmt='png')

In [5]:
import fitz
os.mkdir("/kaggle/working/LLM_RAG_Data/Data_image/")
path='/kaggle/working/LLM_RAG_Data/Data/'
path1='/kaggle/working/LLM_RAG_Data/Data_image/'
doc = fitz.open(path+'ConceptsofBiology.pdf')
seen_xrefs = set()
for page_num in range(doc.page_count):
    for img in doc.get_page_images(page_num):
        xref = img[0]
        if xref in seen_xrefs:
            continue
        image = doc.extract_image(xref)
        imgfile = path1+f"img{xref:05d}.{image['ext']}"
        with open(imgfile, "wb") as fout:
            fout.write(image["image"])
        seen_xrefs.add(xref)
        #print(f"Page {page_num}: {imgfile} ({image['width']} x {image['height']}")

In [6]:
from langchain.docstore.document import Document
texts1=list()
path='/kaggle/working/LLM_RAG_Data/Raw_Data/'
path1='/kaggle/working/LLM_RAG_Data/Data_image/'
from pytesseract import pytesseract
im=[path+i for i in os.listdir(path)]
im.extend([path1+i for i in os.listdir(path1)])
for image_name in im:
    if (image_name.endswith(".jpeg")|image_name.endswith(".png")):
        texts1.append(pytesseract.image_to_string(image_name))
texts1='\n\n'.join(texts1)
texts2=texts1.split('\n\n')
texts=list()
for i in texts2:
    if len(i)>20:
        texts.append(Document(page_content=i.replace('\n',' '), metadata={"source": "local"}))

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
# Define the path to the pre-trained model you want to use
modelPath = "BAAI/bge-small-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':device}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-03 08:08:03.983786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 08:08:03.983908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 08:08:04.221532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
import os
import uuid
#os.mkdir("/kaggle/working/LLM_RAG_Data/VectorDb/")
path='/kaggle/working/LLM_RAG_Data/VectorDb/'
path1='/kaggle/working/LLM_RAG_Data/VectorDb1/'
path2='/kaggle/working/LLM_RAG_Data/Data_image/'
import chromadb
import numpy as np
from langchain.vectorstores import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from PIL import Image as _PILImage
db1 = Chroma.from_documents(texts, embeddings,persist_directory=path)
db2 = Chroma(
    collection_name="mm_rag_clip_photos", embedding_function=OpenCLIPEmbeddings(),persist_directory=path1)
# Get image URIs with .jpg extension only
image_uris = sorted(
    [
        os.path.join(path2, image_name)
        for image_name in os.listdir(path2)
        if image_name.endswith(".jpeg")
    ]
)

# Add images
db2.add_images(uris=image_uris)
# Make retriever

['26dc64e2-e7b8-4cba-a95c-cbd3561af210',
 'dd3e5d5d-a1a8-4309-b7e4-f8f28135f814',
 '32af36c6-62d4-4e81-ad59-3a3150d3525b',
 '209b7788-a9cb-418a-8ee8-a66afbb7c3ee',
 '77500531-ea2f-441d-b171-92d0fdc66cde',
 '39d7a3da-1d25-4f81-bd36-a00c023a4365',
 '06f52716-483e-4c6a-b043-53534f7cf8d9',
 '53b47d5c-751f-45cb-9c83-5f5cb9480a78',
 '395a64d1-e7e5-4c62-aba0-a4e1b3a92a85',
 '5e3d51d8-b962-40df-abf2-6021ab235186',
 'e3f7ef7a-90fe-4e3c-a877-424f44781ecc',
 'd66e2f11-a78b-4415-841b-def2ae99e236',
 '50be1cf0-8234-4b11-bacf-8aaa76f3bcd7',
 'fc59ae34-bdac-45e7-a507-f023f72a0a06',
 '763d8ebf-62cb-4e88-bd0e-057d3b4dcac6',
 'de3773ea-d8f6-4557-98af-df38f22ca40f',
 '9dc54a51-5259-4f46-8190-bca4a26f8a9e',
 'faa2e811-2935-4563-9d90-361a1026a0b9',
 'b7783124-aa2c-4772-9fed-ec3b9ce61800',
 '82445a56-0cc1-4d6e-af12-0c20fc4b1922',
 '36ef4bd9-0ccf-4104-b52c-b9765bf9e025',
 '243e31d2-bbc0-4f3c-b9d0-ebf4e43ff754',
 '57156f2f-a3a1-4b5e-8374-d2cd82bd1117',
 '72767164-7193-4263-a6be-4776156c5f1f',
 '4dbdc582-42da-

In [11]:
db1.persist()
db2.persist()


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [15]:
# db1=Chroma(persist_directory=path,embedding_function=embeddings)
# db1.get()
# db2=Chroma(persist_directory=path1,collection_name="mm_rag_clip_photos", embedding_function=OpenCLIPEmbeddings())
# db2.get()

In [20]:
retriever1 = db1.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever2 = db2.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever_infos = [
    {"name": "retriever1", "description": "Description for retriever1", "retriever": retriever1},
    {"name": "retriever2", "description": "Description for retriever2", "retriever": retriever2},
    # Add more retrievers as needed
]

In [21]:
# Specify the model name you want to use
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, device_map="auto",quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
pipeline_inst = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)
llm = HuggingFacePipeline(pipeline=pipeline_inst,
    model_kwargs={"device_map": device},
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [22]:
pipeline_inst.save_pretrained('/kaggle/working/LLM_RAG_Data/Model')

In [23]:
pipe=pipeline("text-generation", model = '/kaggle/working/LLM_RAG_Data/Model')

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [31]:
from langchain.chains.router.multi_retrieval_qa import MultiRetrievalQAChain
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.schema import BaseRetriever
from operator import itemgetter
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough,RunnableParallel
from langchain.chains import TransformChain,LLMChain,RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

#default_retriever = BaseRetriever()

# Create the MultiRetrievalQAChain

QA_Prompt=PromptTemplate(input_variables=["context", "question"],
template= """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:""",)
# RAG pipeline
# chain = MultiRetrievalQAChain.from_retrievers(
#     llm=llm,
#     retriever_infos=retriever_infos,
#     default_retriever=default_retriever,
#     default_prompt=QA_Prompt,
#     return_source_documents=True
# )
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever1,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_Prompt}
)

In [32]:
response = chain.invoke({'query':"Define alternative RNA splicing?"})

In [33]:
response['result']

' Alternative RNA splicing refers to the process by which different protein products can be generated from a single gene through the selective inclusion or exclusion of certain sections of non-coding DNA called introns during the processing of messenger RNA (mRNA) molecules. This mechanism allows for greater complexity and versatility in gene expression, as different combinations of exons (coding regions) can result in the production of distinct protein isoforms with varying functions. Alternative splicing is a common feature of eukaryotic gene regulation, with estimates suggesting that over 70% of human genes undergo this process.'

In [34]:
response['source_documents']

[Document(page_content='Alternative RNA Splicing', metadata={'source': 'local'}),
 Document(page_content='Alternative RNA Splicing', metadata={'source': 'local'}),
 Document(page_content='Alternative RNA Splicing', metadata={'source': 'local'}),
 Document(page_content='In the 1970s, genes were first observed that exhibited alternative RNA splicing. Alternative RNA splicing is a mechanism that allows different protein products to be produced from one gene when different combinations of introns (and sometimes exons) are removed from the transcript (Figure 9.23). This alternative splicing can be haphazard, but more often it is controlled and acts as a mechanism of gene regulation, with the frequency of different splicing alternatives controlled by the cell as a way to control the production of different protein products in different cells, or at different stages of development. Alternative splicing is now understood to be a common mechanism of gene regulation in eukaryotes; according to o

#Chunking and vector embedding can be done in the following way as well. But, the following way doesn't change the tables properly.

#Future tasks:
* There are images as well in the books. Images are fetched.
* Images should be titled in the way so that when we will as any question then the images can also be retrieved along with answer
* We could have used Multimodel or used Huggingface Image-to-text models if we have to retrieve the information from the images. These information will be appended to the context and passed in the prompt.
* I was trying to use the Multiretriever for using the images


In [ ]:
# # Extract images, tables, and chunk text
# from unstructured.partition.pdf import partition_pdf

# raw_pdf_elements = partition_pdf(
#     filename='/kaggle/working/LLM_RAG_Data/Data/ConceptsofBiology.pdf',
#     extract_images_in_pdf=True,
#     infer_table_structure=True,
#     chunking_strategy="by_title",
#     max_characters=4000,
#     new_after_n_chars=3800,
#     combine_text_under_n_chars=2000,
#     image_output_dir_path=path)

In [ ]:
# tables = []
# texts = []
# for element in raw_pdf_elements:
#     if "unstructured.documents.elements.Table" in str(type(element)):
#         tables.append(str(element))
#     elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
#         texts.append(str(element))
# #
# print(len(tables))
# print(len(texts))

In [ ]:
# import os
# import uuid

# import chromadb
# import numpy as np
# from langchain.vectorstores import Chroma
# from langchain_experimental.open_clip import OpenCLIPEmbeddings
# from PIL import Image as _PILImage

# # Create chroma
# vectorstore = Chroma(
#     collection_name="mm_rag_clip_photos", embedding_function=OpenCLIPEmbeddings(),persist_directory=path
# )

# # Get image URIs with .jpg extension only
# image_uris = sorted(
#     [
#         os.path.join(path, image_name)
#         for image_name in os.listdir(path)
#         if image_name.endswith(".jpg")
#     ]
# )

# # Add images
# vectorstore.add_images(uris=image_uris)

# # Add documents
# vectorstore.add_texts(texts=texts)


In [ ]:
# vectorstore.persist()

In [ ]:
# vectorstore=Chroma(persist_directory=path,collection_name="mm_rag_clip_photos",embedding_function=OpenCLIPEmbeddings())
# # vectorstore.get()

In [ ]:
!yes | sudo apt install git-all

In [ ]:
%cd /kaggle/working/LLM_RAG_Data
!git config user.name "Sonal25Sharma"
!git config user.email "eshsonal@gmail.com" 
!git init
!git remote add origin https://github.com/Sonal25Sharma/LLM_RAG_Data.git
!git add .
!git status
!git commit -m "LLM_RAG_Data"
!git config user.name "Sonal25Sharma"
!git config user.email "eshsonal@gmail.com" 
!git checkout -b rag
!git push origin rag --force